In [10]:
from aswe.utils.request import http_request
from datetime import date
from dotenv import load_dotenv
import os

In [11]:
load_dotenv()
headers = {
  'x-rapidapi-key': os.getenv("Sports_API_key"),
  'x-rapidapi-host': 'v1.handball.api-sports.io'
}
print(headers)

{'x-rapidapi-key': '8ecf2f5824f80d65b69e7dc7110e9285', 'x-rapidapi-host': 'v1.handball.api-sports.io'}


In [78]:
def get_league_id(league_name:str) -> int:
    data = http_request(f'https://v1.handball.api-sports.io/leagues', headers=headers).json()
    for league in data['response']:
        if league['name'] == league_name:
            return league['id']
    return None

In [80]:
def get_league_table(league_name = "Bundesliga") -> list:
    league_id = get_league_id(league_name)
    if league_id is None:
        return None
    standings = http_request(f'https://v1.handball.api-sports.io/standings?league={league_id}&season=2022', headers=headers).json()
    table = []
    for position in standings['response'][0]:
        table.append(f'{position["position"]} {position["team"]["name"]} {position["points"]}')
    return table

In [81]:
def get_team_id(team_name: str) -> int:
    team_name = team_name.replace(' ', '%20')
    data = http_request(f'https://v1.handball.api-sports.io/teams?name={team_name}', headers=headers).json()
    
    if data["response"] == []:
        return None
    
    return data["response"][0]["id"]

In [88]:
def get_team_game_today(team_name: str, league_name = "Bundesliga") -> list:
    league_id = get_league_id(league_name)
    if league_id is None:
        return None
    today = date.today()
    team_name = team_name.replace(' ', '%20')
    id = get_team_id(team_name)
    if id == None:
        return None
    data = http_request(f'https://v1.handball.api-sports.io/games?date={today}&timezone=Europe/Berlin&league={league_id}&season=2022&team={id}', headers=headers).json()
    
    if data["response"] == []:
        return None
    games = []
    for game in data["response"]:
        games.append(f"{game['teams']['home']['name']} {game['scores']['home']} - {game['scores']['away']} {game['teams']['away']['name']}")
    return games